In [1]:
!pip install python-dotenv gradio langchain_openai langchain_chroma langchain-community langchain_groq

  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached fastapi-0.116.1-py3-none-any.whl.metadata (28 kB)
  Using cached ffmpy-0.6.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached safehttpx-0.1.6-py3-none-any.whl.metadata (4.2 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached tomlkit-0.13.3-py3-none-any.whl.metadata (2.8 kB)
  Using cached uvicorn-0.35.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\Babji\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [2]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [4]:
 !pip install --upgrade langchain langchain-community


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\Babji\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [8]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.document_loaders import DirectoryLoader, NotebookLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
import getpass
from openai import OpenAI

ModuleNotFoundError: No module named 'langchain_openai'

In [ ]:
from sklearn.manifold import TSNE
import plotly.graph_objects as go
import matplotlib.pyplot as plt

import numpy as np

In [ ]:
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")
llm = ChatGroq(model="llama-3.1-8b-instant",temperature=0.2)


Enter your Groq API key: ··········


In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API KEY:")

load_dotenv(override=True)
OPENAI_KEY = os.getenv('OPENAI_API_KEY')

#sk-proj-U91Ha3fyjkNt92FCBs2WFH-wWpP_musp4c-4Lj4zWMjUjN7zbIciza4Bov8UHS50eVQB4W_BrWT3BlbkFJVSsk9EGo0_Ha310lwCKm5RqDIs_iccEXQ2mdakiIDunIjTCbW9ndn8lKzdG2nh1TgGFN4quWIAST
# MODEL = 'gpt-4o-mini'
# openai = OpenAI()

Enter OpenAI API KEY:··········


In [ ]:
# Colab defaults
NOTEBOOKS_DIR = '/content/ColabNotebooks'
DB_DIR = '/content/colab_vector_db'

In [ ]:
# Read the product markdown notebooks from the colab

notebooks = glob.glob(
    os.path.join(NOTEBOOKS_DIR, "**", "*.md"),
    recursive=True
)
print(f"Notebooks found: {len(notebooks)}")


loader = DirectoryLoader(NOTEBOOKS_DIR,
                         glob = '**/*.md',
                         loader_cls = TextLoader
                        )
docs = loader.load()

Notebooks found: 4


In [ ]:
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

splitter = RecursiveCharacterTextSplitter(
    chunk_size = CHUNK_SIZE,
    chunk_overlap = CHUNK_OVERLAP,
    separators=["\n## ", "\n### ", "\n#### ", "\n\n", "\n", " ", ""]
)

chunks = splitter.split_documents(docs)
print(f'Created {len(chunks)} chunks from your notebooks')

Created 22 chunks from your notebooks


In [ ]:
# Create OpenAI Embeddings

from langchain_openai import OpenAIEmbeddings


embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)



if os.path.exists(DB_DIR):

    Chroma(persist_directory = DB_DIR, embedding_function = embeddings).delete_collection()



vectorstore = Chroma.from_documents(
    documents = chunks,
    embedding = embeddings,
    persist_directory = DB_DIR
)

vector_count = vectorstore._collection.count()
print(f"Vectorstore contains {vector_count} vectors.")

Vectorstore contains 22 vectors.


In [ ]:
sample_embedding = vectorstore._collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"There are {vectorstore._collection.count():,} vectors with {dimensions:,} dimensions in the vector store.")

There are 22 vectors with 1,536 dimensions in the vector store.


In [ ]:
result = vectorstore._collection.get(include=['embeddings', 'documents',])
vectors = np.array(result['embeddings'])
documents = result['documents']
colors = ['blue'] * len(vectors)

In [ ]:
tsne = TSNE(n_components=3, random_state=42, perplexity=min(22 - 1, 30))
reduced_vectors = tsne.fit_transform(vectors)

fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=4, color=colors, opacity=0.8),
    text=[d[:100] + "..." for d in documents],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D TSNE of Notebook-Chunks',
    scene=dict(
        xaxis_title="TSNE-1",
        yaxis_title="TSNE-2",
        zaxis_title="TSNE-3"
    ),
    width=800,
    height=600,
    margin=dict(r=10, b=10, l=10, t=40)
)

fig.show()

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 4}),
    return_source_documents=True
)

In [ ]:
memory = ConversationBufferMemory(
    memory_key = 'chat_history',
    return_messages = True
)

conv_chain = ConversationalRetrievalChain.from_llm(
    llm = llm,
    retriever = vectorstore.as_retriever(search_kwargs = {'k': 10}),
    memory = memory
)

<ipython-input-14-55d2ff9f6153>:1: LangChainDeprecationWarning:

Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/



In [ ]:
def chat_with_memory_and_sources(message, chat_history):
    # Get the conversational answer (memory included)
    conv_res = conv_chain.invoke({
        "question":     message,
        "chat_history": chat_history
    })
    answer = conv_res["answer"]

    # Retrieve source documents
    src_res  = qa({"query": message})
    src_docs = src_res["source_documents"]

    # Extract and dedupe product filenames from metadata
    notebooks = [
        os.path.basename(doc.metadata.get("source", ""))
        for doc in src_docs
        if doc.metadata.get("source")
    ]
    unique = []
    for n in notebooks:
        if n not in unique:
            unique.append(n)

    # Append the list of notebook filenames
    if unique:
        answer += "\n\n**Found Notebooks:**\n" + "\n".join(f"- {n}" for n in unique)
    else:
        answer += "\n\n_No Notebooks found._"

    return answer

In [ ]:
view = gr.ChatInterface(chat_with_memory_and_sources,
                        title="Intelligent-RAG-Assistant",
                       type = 'messages').launch(inbrowser=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://193445f8500e85efc0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
